# Finding Label Errors in Object Detection (object detection) Datasets

This 5-minute quickstart tutorial demonstrates how to find potential label errors in object detection datasets. In these datasets each example contains a bounding box and a class label surrounding a physical object within an image scene. Using this labeled data, we train a model to predict classes of objects in an image and their physical locations. The example notebook used to train the model is avalible here (link).

This tutorial uses a subset of the [COCO (Common Objects in Context)](https://cocodataset.org/#home) dataset with 5 classes: car, chair, cup, person, traffic light. The images in the dataset are taken from everyday scenes, and the given annotations come from both official COCO and SAMA.

#### Overview of what we we'll do in this tutorial
- Score examples based on their overall label quality score using `cleanlab.object_detection.rank.get_label_quality_scores`
- Find examples with label issues using `cleanlab.object_detection.filter.find_label_issues`
- Visualize examples using `cleanlab.object_detection.summary.visualize`

<div class="alert alert-info">
Quickstart
<br/>
    
Already have `labels` and `predictions` in the proper format? Just run the code below to find label issues in your object detection dataset.


<div  class=markdown markdown="1" style="background:white;margin:16px">  
    
```python

from cleanlab.object_detection.rank import find_label_issues
from cleanlab.object_detection.filter import get_label_quality_scores

# To get boolean vector of label issues for all images
is_label_issue = find_label_issues(labels, predictions)

# To get label quality scores for all images
label_quality_scores = get_label_quality_scores(labels, predictions)
    
    
```

</div>
</div>

# 1. Install required dependencies and download data
You can use `pip` to install all packages required for this tutorial as follows
```
!pip install cleanlab
!pip insrall matplotlib
```

In [ ]:
!wget -nc 'https://cleanlab-public.s3.amazonaws.com/ObjectDetectionBenchmarking/tutorial_obj/predictions.pkl'
!wget -nc 'https://cleanlab-public.s3.amazonaws.com/ObjectDetectionBenchmarking/tutorial_obj/labels.pkl'
!wget -nc 'https://cleanlab-public.s3.amazonaws.com/ObjectDetectionBenchmarking/tutorial_obj/example_images.zip' && unzip -q -o example_images.zip

In [ ]:
import pickle

from cleanlab.object_detection.rank import get_label_quality_scores, issues_from_scores
from cleanlab.object_detection.filter import find_label_issues
from cleanlab.object_detection.summary import visualize

# 2. Format data, labels and model predictions

In object detection datasets, each given `label` is a made up of bounding box coordinates and a class label. A model `prediction` is also made up of a bounding box and class label as well as the model confidence in the prediction (i.e. probability the prediction is correct). For both `get_label_quality_scores` and `find_label_issues` cleanlab requires a list of given labels and a list of model predictions for all examples.

The expected format of these `labels` and `predictions` is modeled after what common models like mmdet and detectron2 expect. 

The `5` possible class labels in our dataset are: `[car, chair, cup, person, traffic light]`. These classes are zero-indexed and represented with an integer 0,1,...,4.

`labels` is a list of dictionaries where `labels[i]` is a dictionary containing key `labels`, which is a list of class labels corresponding key `bboxxes`, which is a numpy array of bounding boxes. Each bounding box in `labels[i]['bboxes']` is in the format is in the format ``[x1,y1,x2,y2]`` where `(x1,y1)` corresponds to the bottom-left corner of the box and `(x2,y2)` corresponds to the top-right. An example output of `labels[i]` is below:

```
{'bboxes': array([[ 26.5 , 215.25,  88.  , 229.75],
        [116.5 , 189.57, 166.5 , 215.07],
        [241.95, 180.42, 293.32, 225.82]], dtype=float32),
 'labels': array([1, 1, 1]), 
 ...
 'seg_map': '000000037777.png'} 
```

`predictions` is a list of numpy arrays where `predictions[i]` is of shape `(M,)` where `M` is the number of classes and `predictions[i][m]` is of shape `(L,5)` where `L` is the number of bounding boxes for class `k` and the five columns correspond to ``[x1,y1,x2,y2,pred_prob]`` returned by the model. Here `pred_prob` is the model confidence in the predicted label. Our example uses `K == 5` classes which is why `predictions[0].shape = (5,)`.

An example output of `predictions[i]` is below:

```
array([array([], shape=(0, 5), dtype=float32),
       array([[2.41300339e+02, 1.77116043e+02, 2.97747314e+02, 2.28405350e+02,
               1.15757108e-01],
              [8.42839813e+01, 1.88506973e+02, 1.69366867e+02, 2.28179962e+02,
               1.06976844e-01]], dtype=float32)                               ,
       array([[2.09936218e+02, 1.22989265e+02, 2.15952652e+02, 1.33577789e+02,
               2.66048908e-01],
              [3.35775848e+02, 5.53170547e+01, 3.52000000e+02, 7.79105530e+01,
               6.99099600e-02]], dtype=float32)                               ,
       array([], shape=(0, 5), dtype=float32),
       array([], shape=(0, 5), dtype=float32)], dtype=object)
```

Once you have pred_probs and labels in the appropriate formats, you can **find label issues with cleanlab for any object detection dataset**!

The 5 possible class labels in our dataset are car, chair, cup, person, traffic light. These classes are zero-indexed and represented with an integer 0,1,...,4. 



In [ ]:
predictions = pickle.load(open("predictions.pkl", "rb"))
labels = pickle.load(open("labels.pkl", "rb"))

Here’s what these look like for an example in our synthetic object detection dataset.

In [ ]:
example_idx = 8
predictions[example_idx]

In [ ]:
labels[example_idx]

We can visualize the bounding boxes and labels using `cleanlabs.object_detection.summary.visualize` function. Here the given labels are in red and the predicted in blue.

In [ ]:
image_path = 'example_images/' + labels[example_idx]['seg_map']

visualize(image_path, label=labels[example_idx], prediction=predictions[example_idx])

# 3. Use cleanlab to find label issues
Based on the given `labels` and model `predictions` from our trained model, cleanlab can quickly help us find examples that contain label errors in the dataset. In object detection, label errors are annotations that are more likely imperfect.

Examples are imperfect when annotators:
- overlooked an object (missing annotated bounding box),
- chose the wrong class label for an annotated box in the correct location,
- imperfectly annotated the location/edges of a bounding box.


For cleanlab, any of these annotation errors should lead to an image with a lower label quality score.

In [ ]:
num_examples_to_show = 5

In [ ]:
label_issue_idx = find_label_issues(labels, predictions, return_indices_ranked_by_score=True)
label_issue_idx[:num_examples_to_show]

In [ ]:
is_label_issue = find_label_issues(labels, predictions)
is_label_issue[:num_examples_to_show]

## Label quality scores
The above example shows how to identify which examples have label quality issues. We can also compute scores for each example in the dataset which estimate our confidence that this example has been correctly labeled. These scores range between 0 and 1 with smaller values indicating examples whose annotation seems more likely to be imprefect.

Note: This method is useful for prioritizing which images to review if you have too little time to review all of the label issues by reviewing the images with the lowest scores first.

In [ ]:
scores = get_label_quality_scores(labels, predictions)
scores[:num_examples_to_show]

We can also use **issues_from_scores()** which returns an array of issue indices sorted from most to least severe who's label quality scores fall below the threshold if one is provided.

In [ ]:
issue_idx = issues_from_scores(scores, threshold=0.5)
issue_idx[:num_examples_to_show], scores[issue_idx][:num_examples_to_show]

# 4. Visualize Label Issues
Finally, we can use cleanlab functionality to visualize the potential issue errors. Lets take a closer look at an example with a low score using cleanlab's `visualize()` function. To make analysis easier we can also input a `class_names` dictionary into the function to show up as a legend and turn off `overlay` to see the given and predicted labels side by side.

In [ ]:
example_issue_idx = issue_idx[10]

# create a class_labels dictionary to pass in and see labels in the legend
class_names = {"0": "car", "1": "chair", "2": "cup", "3":"person", "4": "traffic light"}

label = labels[example_issue_idx]
prediction = predictions[example_issue_idx]
score = scores[example_issue_idx]
image_path = 'example_images/' + label['seg_map']

print(image_path, '| idx', example_issue_idx , '| label quality score:', score, '| is issue: True')
visualize(image_path, label=label, prediction=prediction, class_names=class_names, overlay=False)

As you can see the given label is marked in a solid red color on the image to the left. This is the provided annotation in which issues are detected in. The model predictions, or the `predicted label` is in blue and overlayed on the image on the right. Each bounding box contains a class number in the top corner (left for given annotations, right for predicted) specific to which class that bounding box was marked as. With our provided `class_names` dictionary we can easily identify which classes were marked in the legend to the right of both images.

This image has a **low** label quality score and is marked as an error. On closer inspection we notice the annotator missed a car in the back right corner that the model identified. Additionally, the bounding box for the foreground car the woman is in is cut short a bit on the right. 

Since our model is not perfect, it incorrectly identified the passenger dog as a person however it was still able to identify issues in the scene. Better trained models will lead to better label error detection but you don't need a near perfect model to identify label issues.


### Different kinds of label issues identified by ObjectLab
Now lets view the first 5 images in our test dataset that are clearly marked as issues and see what other inconsistencies between the `given` and `predicted` label we can spot.

In [ ]:
example_issue_idx = issue_idx[:num_examples_to_show]

issue_predictions = [predictions[idx] for idx in example_issue_idx]
issue_labels = [labels[idx] for idx in example_issue_idx]
issue_scores = scores[example_issue_idx]

for idx, (prediction,label,score) in enumerate(zip(issue_predictions,issue_labels,issue_scores)):
    image_path = 'example_images/' + label['seg_map']
    print(image_path, '| idx', example_issue_idx[idx] , '| label quality score:', score, '| is issue: True')
    visualize(image_path, label=label, prediction=prediction, class_names=class_names, overlay=False)

In each of these these examples there is a clear variety of issues. The corn-on-the-cob has a swapped label where a chair was identified as a person while the monitor picture is missing annotations for a cup and a person working on a chair in the background. The annotator of a kid's birthday party did not individually highlight each kid (in accordance to COCO guidelines only groups with 10 or more objects of the same type can be identified with a single box as a "crowd"). Some cups were forgotten at the dinner party and the empty chair was annotated incorrectly. Finally, the chairs are missing from the foreground of the abandoned school bus. 

All of these examples has low label quality scores to reflect their low annotation quality.

### Other uses of visualize
The visualize function is also equally able to visualize non-issues, labels, predictions or just the image seperately and save the created images. Lets take a look at the first couple of images in our dataset with this functionality.

We can save the image by providing a `save_path` variable as with `idx=0`. Notice how the label quality score is high for this example and it is marked as a non issue. Both the given and predicted labels closeley resemble each other contributing to the high score.

For `idx=1`, notice how we are only passing in the given annotations to visualize. Similarily for `idx=2` we are just looking at the image.

In [ ]:
idx = 0
image_path = 'example_images/' + labels[idx]['seg_map']
print(image_path, '| idx', idx , '| label quality score:', scores[idx], '| is issue:', is_label_issue[idx])
visualize(image_path, label=labels[idx], prediction=predictions[idx], class_names=class_names, save_path='./example_image.png')

In [ ]:
idx = 1
image_path = 'example_images/' + labels[idx]['seg_map']
print(image_path, '| idx', idx , '| label quality score:', scores[idx], '| is issue:', is_label_issue[idx])
visualize(image_path, label=labels[idx], class_names=class_names)

In [ ]:
idx = 2
image_path = 'example_images/' + labels[idx]['seg_map']
visualize(image_path)

Here we notice examples where the predictions and labels are more similar have higher quality scores than those that are missmatched, and are less likeley to be marked as issues. The number of boxes is agnostic to the score, consider the three cross country skiiers recieving a similar score to a single skiier in a similar enviornment.

On the contrary, images with missing annotated boxes, or boxes in different locations recieve lower scores.

In [ ]:
# Note: This cell is only for docs.cleanlab.ai, if running on local Jupyter or Colab, please ignore it.
import numpy as np

assert set([60, 48, 1, 46, 30]).issubset(issue_idx)
assert 2 not in issue_idx and 0 not in issue_idx